- Flood Frequency 
- Flood Pronearea
- Slope Flat area from SRTM
- Montly water surface layer by Time period 

In [2]:
import ee
import geemap
import datetime
import ipywidgets as widgets
from IPython.display import display

# Initialize Earth Engine API
ee.Initialize()

# Create map
Map = geemap.Map()

# Define region of interest (ROI) for Guyana
gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Timor-Leste'))
georgetown = guyana.filter(ee.Filter.eq('ADM1_NAME', 'Georgetown'))
roi = guyana.geometry()

# Define dynamic time period
start_date = datetime.date(1984, 3, 16)
end_date = datetime.date(2024, 12, 31)

# Convert to ee.Date objects
start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# Load datasets
jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
srtm = ee.Image("USGS/SRTMGL1_003")

# Filter JRC dataset for the selected time period and region
jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)

# Step 1: Exclude permanent water bodies
permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))

# Step 2: Calculate Flood Frequency
def calculate_flood_frequency(collection):
    # Detect observations and water occurrences
    def add_bands(img):
        obs = img.gt(0).rename("obs")
        water = img.select("water").eq(2).rename("water")
        return img.addBands([obs, water])
    
    collection = collection.map(add_bands)
    total_obs = collection.select("obs").sum().rename("total_obs")
    total_water = collection.select("water").sum().rename("total_water")
    flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
    return flood_frequency.updateMask(flood_frequency.neq(0))

flood_frequency = calculate_flood_frequency(jrc_filtered)

# Step 3: Analyze DEM
# Calculate slope and flat areas
slope = ee.Terrain.slope(srtm).rename("slope")
flat_area = slope.lt(5).rename("flat_area")  
low_lying = srtm.lt(10).rename("low_lying")  

# Step 4: Flood-Prone Areas
flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))

# Step 5: Monthly Water Surface Maps
def get_monthly_flood_layers(collection, roi, permanent_water):
    """
    Extracts monthly flood layers from the given collection,
    excluding permanent water bodies.
    """
    monthly_layers = {}
    months = collection.aggregate_array("system:index").getInfo()
    
    for month in months:
        img = collection.filter(ee.Filter.eq("system:index", month)).first()
        water_layer = img.select("water").eq(2).updateMask(permanent_water.Not())
        monthly_layers[month] = water_layer.clip(roi)
    
    return monthly_layers

# Call the function with the permanent water exclusion
monthly_layers = get_monthly_flood_layers(jrc_filtered, roi, permanent_water)

# Visualization parameters
dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
flat_area_vis = {"min": 0, "max": 1, "palette": ["ffffff", "0000ff"]}
flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}
flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
water_surface_vis = {"min": 0, "max": 1, "palette": ["0000ff"]}

# Interactive widgets
start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
run_button = widgets.Button(description="Run Analysis", button_style="success")

# # Function to update map after running analysis
# def update_map(change):
#     start_date = start_date_picker.value
#     end_date = end_date_picker.value

#     # Convert to ee.Date objects
#     start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
#     end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

#     # Re-filter the JRC dataset with the new date range
#     jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
#     # Exclude permanent water bodies
#     jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))
#     # Calculate flood frequency
#     flood_frequency = calculate_flood_frequency(jrc_filtered)
#     # Calculate flood-prone areas
#     flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))
    

#     # Update layers
#     Map.layers = []  # Clear current layers
#     Map.centerObject(roi, 8)
#     Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
#     Map.addLayer(slope.clip(roi), slope_vis, "Slope")
#     Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, f"Flood Frequency from {start_date} to {end_date}")
#     Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, f"Flood-Prone Areas from {start_date} to {end_date}")

def update_map(change):
    start_date = start_date_picker.value
    end_date = end_date_picker.value

    # Convert to ee.Date objects
    start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
    end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

    # Re-filter the JRC dataset with the new date range
    jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
    # Exclude permanent water bodies
    jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))
    # Calculate flood frequency
    flood_frequency = calculate_flood_frequency(jrc_filtered)
    # Calculate flood-prone areas
    flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))
    
    # Add new layers without removing existing ones
    Map.centerObject(roi, 8)
    Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
    Map.addLayer(slope.clip(roi), slope_vis, "Slope")
    Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, f"Flood Frequency from {start_date} to {end_date}")
    Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, f"Flood-Prone Areas from {start_date} to {end_date}")
   
    # for month, layer in monthly_layers.items():
    #     Map.addLayer(layer, water_surface_vis, f"Water Surface {month}")

# Set the button action
run_button.on_click(update_map)

# Display the widgets
display(start_date_picker, end_date_picker, run_button)

# Show the map
Map.addLayerControl()
Map


DatePicker(value=datetime.date(1984, 3, 16), description='Start Date', step=1)

DatePicker(value=datetime.date(2024, 12, 31), description='End Date', step=1)

Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…